# Applied Data Science - Capstone Project

## Install required Python packages

In [43]:
!conda install -c conda-forge geopy --yes 
!conda install -c conda-forge folium --yes 
!conda install -c conda-forge pyquery --yes

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



## Get Wiki page containing Toronto Boroughs/Neighborhoods
### Note: using pandas.io.hmtl to get the wiki table into pandas DataFrame

In [204]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
wikitables = soup.find_all('table') 
Toronto = pd.read_html(str(wikitables[0]), index_col=None, header=0)[0]
Toronto.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [205]:
# TO VERIFY THAT BOTH METHODS PRODUCE SIMILAR DIMENSIONS
Toronto.shape

(289, 3)

## Alternative way to read in the content and produce a dataframe using pandas.io.html
### Note: Results are consistent between the two methods

In [22]:
import requests
import numpy as np
import pandas as pd
from pandas.io.html import read_html

# Define the wiki page url var
WIKI_URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
# Issue HTTP request to get the URL content
req = requests.get(WIKI_URL)
# Use pandas read_html to read in the content
wikitables = read_html(WIKI_URL, index_col=None, header=0, attrs={"class":["sortable","wikitable"]})
# Get pandas dataframe
Toronto = wikitables[0]
Toronto.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [207]:
# TO VERIFY THAT BOTH METHODS PRODUCE SIMILAR DIMENSIONS
Toronto.shape

(289, 3)

## Data cleaning

In [23]:
# Empty entries to np.nan to drop them in the next step
Toronto['Borough'].replace('', np.nan, inplace=True)
# Drop np.nan to remove rows not containing meaningful data
Toronto.dropna(subset=['Borough'], inplace=True)
# Leave behind rows containing 'Not assigned' in 'Borough'
Toronto = Toronto[Toronto['Borough'] != 'Not assigned']

## Data processing - 'Not assigned' to value

In [24]:
# Iterate over the dataframe and fix 'Not assigned' for column 'Neighborhood'
for i, _ in Toronto.iterrows():
    if Toronto.loc[i]['Neighbourhood'] == 'Not assigned': Toronto.loc[i]['Neighborhood'] = Toronto.loc[i]['Borough']

## Dataframe shape

In [25]:
# Check datafame shape
Toronto.shape

(212, 3)

## Number of rows in the dataframe

In [26]:
# Print the number of rows in the dataframe
print('Number of rows in Toronto dataframe: {}'.format(Toronto.shape[0]))

Number of rows in Toronto dataframe: 212


In [32]:
!conda install -c conda-forge geocoder --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/ibm/conda/miniconda3

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ratelim-0.1.6              |           py35_0           5 KB  conda-forge
    orderedset-2.0             |           py35_0         685 KB  conda-forge
    certifi-2018.8.24          |           py35_1         138 KB  conda-forge
    geocoder-1.38.1            |             py_0          52 KB  conda-forge
    ca-certificates-2018.8.24  |       ha4d7672_0         136 KB  conda-forge
    openssl-1.0.2p             |       h470a237_0         3.5 MB  conda-forge
    conda-4.5.11               |           py35_0         636 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         5.1 MB

The following NEW packages will be INS

## API compensator - at times geocoder.google returns None for the same postal code
### Create a dictionary of all the postal codes to add to the dictionary at the next step

In [27]:
TPS = Toronto['Postcode'].unique()
len(TPS)

103

## Get offline cache to support throttled Geocoder API

In [28]:
!wget -q --no-check-certificate -O 'latitude.pickle' 'https://docs.google.com/uc?export=download&id=1PdEOkPErrpBtDgSlDwczIv_KLlpY-YcO'
!wget -q --no-check-certificate -O 'longitude.pickle' 'https://docs.google.com/uc?export=download&id=1XujA04dCARQnlxu-X2ItOVcYQz0MMQh9'

In [29]:
!ls -l *.pickle

-rw-r--r-- 1 spark spark 1965 Sep 12 18:44 latitude.pickle
-rw-r--r-- 1 spark spark 1965 Sep 12 18:44 longitude.pickle


In [30]:
import pickle

with open('latitude.pickle', 'rb') as flat:
    latitude = pickle.load(flat)
with open('longitude.pickle', 'rb') as flon:
    longitude = pickle.load(flon)

In [33]:
import geocoder
import time

for postcode in TPS:
    # When offline cache is available use it to avoid Geocoder Google API throttling
    if postcode in latitude.keys(): 
        continue
    while True:
        g = geocoder.google('{}, Toronto, Ontario'.format(postcode))
        lat_lng_coords = g.latlng
        if lat_lng_coords == None:
            print('Trottled response to {}'.format(postcode))
            time.sleep(5)
            continue
        break
    
    if lat_lng_coords != None:
        latitude[postcode] = lat_lng_coords[0]
        longitude[postcode] = lat_lng_coords[1]
print('Successfully populated geo locations')

Successfully populated geo locations


In [34]:
lat = []
lon = []
for i, _ in Toronto.iterrows():
    lat.append(latitude[Toronto.loc[i]['Postcode']])
    lon.append(longitude[Toronto.loc[i]['Postcode']])
                        

In [35]:
Toronto = Toronto.assign(Latitude = lat, Longitude=lon)
Toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
5,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
6,M6A,North York,Lawrence Heights,43.718518,-79.464763


In [36]:
Toronto = Toronto.reset_index(drop=True)
Toronto.to_csv('Toronto.csv')
Toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


## Additional imports

In [219]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Import Demographics Data

### Pull the data from Wikipedia

In [43]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

res = requests.get("https://en.wikipedia.org/wiki/Demographics_of_Toronto_neighbourhoods")
soup = BeautifulSoup(res.content,'lxml')
wikitables = soup.find_all('table') 
Toronto_Demographics = pd.read_html(str(wikitables[1]), index_col=None, header=0)[0]

### Post process the dataframe Toronto_Demographics

In [44]:
columns_to_drop = [
    'FM', 
    'Census Tracts', 
    '% Change in Population since 2001', 
    'Land area (km2)', 
    'Transit Commuting\xa0%', 
    '% Renters', 
    'Second most common language (after English) by name', 
    'Map'
]

Toronto_Demographics.drop(columns=columns_to_drop, inplace=True)

Toronto_Demographics = Toronto_Demographics.rename(columns={'Second most common language (after English) by percentage':'ESL'})
Toronto_Demographics = Toronto_Demographics.rename(columns={'Name':'Neighbourhood'})
Toronto_Demographics.drop([0], inplace=True)

Toronto_Demographics.head()
Toronto_Demographics['Percentage'] = Toronto_Demographics.apply (lambda row: row['ESL'].split('% ')[0],axis=1)
Toronto_Demographics['Language'] = Toronto_Demographics.apply (lambda row: row['ESL'].split('% ')[1],axis=1)
Toronto_Demographics.drop(columns=['ESL'], inplace=True)

### Show post processed result on Toronto_Demographics

In [45]:
Toronto_Demographics.head()

,Neighbourhood,Population,Density (people/km2),Average Income,Percentage,Language
1,Agincourt,44577,3580,25750,19.3,Cantonese
2,Alderwood,11656,2360,35239,06.2,Polish
3,Alexandra Park,4355,13609,19687,17.9,Cantonese
4,Allenby,2513,4333,245592,01.4,Russian
5,Amesbury,17318,4934,27546,06.1,Spanish


### Explore the demographics data for the city of Toronto

In [46]:
list_of_neighbourhoods = Toronto_Demographics['Neighbourhood'].values.tolist()

In [47]:
Toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


### Check the neighbourhoods not present in the dataframe Toronto and remove them from the dataframe Toronto_Demographics

In [48]:
list_missing_neigh = []
for neigh in list_of_neighbourhoods:
    if neigh not in Toronto['Neighbourhood'].values and neigh not in Toronto['Borough']:
        list_missing_neigh.append(neigh)
list_missing_neigh

['Alexandra Park',
 'Allenby',
 'Amesbury',
 'Armour Heights',
 'Banbury',
 'Bay Street Corridor',
 'Bayview Woods – Steeles',
 'Bendale',
 'Bloor West Village',
 'Bracondale Hill',
 'Branson',
 'Bridle Path',
 'Caribou Park',
 'Carleton Village',
 'Casa Loma',
 'Centennial',
 'Chaplin Estates',
 'Christie Pits',
 'Clairville',
 'Corktown',
 'Crescent Town',
 'Cricket Club',
 'Davenport',
 'Discovery District',
 'Distillery District/West Don Lands',
 'Don Mills',
 'Don Valley Village',
 'Dovercourt Park',
 'Downsview',
 'Dufferin Grove',
 'Earlscourt',
 'East Danforth',
 'Eatonville',
 'Eglinton East',
 'Elia (Jane and Finch)',
 'Fairbank',
 'Fashion District',
 'Financial District',
 'Forest Hill',
 'Fort York/Liberty Village',
 'Glen Park',
 "Governor's Bridge/Bennington Heights",
 'Graydon Hall',
 'Guildwood',
 'Harbord Village',
 'Harbourfront / CityPlace',
 'Harwood',
 'High Park North',
 'Hillcrest',
 'Hoggs Hollow',
 'Humber Heights',
 'Humber Valley Village',
 'Humbermede',
 'H

In [61]:
Toronto_Demographics = Toronto_Demographics[~Toronto_Demographics.Neighbourhood.isin(list_missing_neigh)]

In [63]:
Toronto_Demographics.reset_index(drop=True)

,Neighbourhood,Population,Density (people/km2),Average Income,Percentage,Language
0,Agincourt,44577,3580,25750,19.3,Cantonese
1,Alderwood,11656,2360,35239,06.2,Polish
2,Bathurst Manor,14945,3187,34169,09.5,Russian
3,Bayview Village,12280,2966,46752,08.4,Cantonese
4,Bedford Park,13749,6057,80827,00.7,Greek
5,Birch Cliff,12266,3525,48965,00.9,Unspecified Chinese
6,Brockton,9039,8217,27260,19.9,Portuguese
7,Cabbagetown,11120,7943,50398,01.6,Unspecified Chinese
8,Church and Wellesley,13397,24358,37653,01.8,Spanish
9,Clairlea,11104,3102,33392,04.3,Filipino


### Drop entries from the dataframe Toronto that are not in the dataframe Toronto_Demographics

In [64]:
list_exist_neigh_dem = Toronto_Demographics['Neighbourhood'].values.tolist()

In [67]:
Toronto = Toronto[Toronto.Neighbourhood.isin(list_exist_neigh_dem)]
Toronto.reset_index(drop=True)
Toronto.shape

(77, 5)

In [66]:
Toronto_Demographics.shape

(76, 6)

In [68]:
Toronto_Demographics.head()

,Neighbourhood,Population,Density (people/km2),Average Income,Percentage,Language
1,Agincourt,44577,3580,25750,19.3,Cantonese
2,Alderwood,11656,2360,35239,06.2,Polish
8,Bathurst Manor,14945,3187,34169,09.5,Russian
10,Bayview Village,12280,2966,46752,08.4,Cantonese
12,Bedford Park,13749,6057,80827,00.7,Greek


In [69]:
Toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
4,M6A,North York,Lawrence Heights,43.718518,-79.464763
5,M6A,North York,Lawrence Manor,43.718518,-79.464763
8,M1B,Scarborough,Rouge,43.806686,-79.194353


### Merge the two dataframes to create a workabable dataframe - Toronto_Merged

In [70]:
Toronto_Merged = pd.merge(Toronto, Toronto_Demographics, on='Neighbourhood')

In [71]:
Toronto_Merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Population,Density (people/km2),Average Income,Percentage,Language
0,M3A,North York,Parkwoods,43.753259,-79.329656,26533,5349,34811,03.4,Unspecified Chinese
1,M4A,North York,Victoria Village,43.725882,-79.315572,17047,3612,29657,03.2,Urdu
2,M6A,North York,Lawrence Heights,43.718518,-79.464763,3769,1178,29867,15.0,Italian
3,M6A,North York,Lawrence Manor,43.718518,-79.464763,13750,6425,36361,07.2,Filipino
4,M1B,Scarborough,Rouge,43.806686,-79.194353,22724,791,29230,15.6,Tamil


In [72]:
Toronto_Merged.groupby('Neighbourhood').info()

AttributeError: Cannot access callable attribute 'info' of 'DataFrameGroupBy' objects, try using the 'apply' method

In [220]:
address = 'Toronto, Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
T_lat = location.latitude
T_lon = location.longitude
print('The geograpical coordinate of Toronto, ON, Canada are {}, {}.'.format(T_lat, T_lon))

/opt/ibm/conda/miniconda3/lib/python3.5/site-packages/geopy/geocoders/osm.py:138: UserWarning: Using Nominatim with the default "geopy/1.16.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto, ON, Canada are 43.653963, -79.387207.


# Create a map of Toronto with neighbourhoods

In [141]:
T_map = folium.Map(location=[T_latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto['Latitude'], Toronto['Longitude'], Toronto['Borough'], Toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(T_map)  
    
T_map

# Prepare Foursquare credentials

In [160]:
CLIENT_ID = '0C4ANZOALOMRDOJZU2BUYUKB2LOF3N5ADH4A3WJ3UDS3YIJD'
CLIENT_SECRET = 'MIJAG0GZX3JF2RXLGPCOOH25QZ3NBF2MLFYQJUG2DD5LCNM3'
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0C4ANZOALOMRDOJZU2BUYUKB2LOF3N5ADH4A3WJ3UDS3YIJD
CLIENT_SECRET:MIJAG0GZX3JF2RXLGPCOOH25QZ3NBF2MLFYQJUG2DD5LCNM3


## Explore West Toronto

In [221]:
HighPark = Toronto[Toronto['Borough'] == 'West Toronto']
HighPark.reset_index(drop=True)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M6H,West Toronto,Dovercourt Village,43.669005,-79.442259
1,M6H,West Toronto,Dufferin,43.669005,-79.442259
2,M6J,West Toronto,Little Portugal,43.647927,-79.419750
3,M6J,West Toronto,Trinity,43.647927,-79.419750
4,M6K,West Toronto,Brockton,43.636847,-79.428191
5,M6K,West Toronto,Exhibition Place,43.636847,-79.428191
6,M6K,West Toronto,Parkdale Village,43.636847,-79.428191
7,M6P,West Toronto,High Park,43.661608,-79.464763
8,M6P,West Toronto,The Junction South,43.661608,-79.464763
9,M6R,West Toronto,Parkdale,43.648960,-79.456325


In [176]:
HP_lat = HighPark['Latitude'].values[0] # neighborhood latitude value
print(type(HP_lat))
HP_lon = HighPark['Longitude'].values[0] # neighborhood longitude value

HP_name = HighPark['Neighbourhood'].values[0] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(HP_name, 
                                                               HP_lat, 
                                                               HP_lon))

<class 'numpy.float64'>
Latitude and longitude values of Dovercourt Village are 43.6690051, -79.4422593.


In [177]:
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    HP_lat, 
    HP_lon, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=0C4ANZOALOMRDOJZU2BUYUKB2LOF3N5ADH4A3WJ3UDS3YIJD&client_secret=MIJAG0GZX3JF2RXLGPCOOH25QZ3NBF2MLFYQJUG2DD5LCNM3&v=20180605&ll=43.6690051,-79.4422593&radius=500&limit=100'

In [178]:
results = requests.get(url).json()
# results
if results != None:
    print("Request successfully processed")

Request successfully processed


In [164]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [179]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The Greater Good Bar,Bar,43.669409,-79.439267
1,Parallel,Middle Eastern Restaurant,43.669543,-79.438701
2,Planet Fitness Toronto Galleria,Gym / Fitness Center,43.667588,-79.442574
3,FreshCo,Supermarket,43.667918,-79.440754
4,Happy Bakery & Pastries,Bakery,43.667050,-79.441791


In [180]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

17 venues were returned by Foursquare.


In [167]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [181]:
# type your answer here
HP_venues = getNearbyVenues(names=HighPark['Neighbourhood'],
                                   latitudes=HighPark['Latitude'],
                                   longitudes=HighPark['Longitude']
                                  )

Dovercourt Village
Dufferin
Little Portugal
Trinity
Brockton
Exhibition Place
Parkdale Village
High Park
The Junction South
Parkdale
Roncesvalles
Runnymede
Swansea


In [182]:
print(HP_venues.shape)
HP_venues.head()

(373, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Dovercourt Village,43.669005,-79.442259,The Greater Good Bar,43.669409,-79.439267,Bar
1,Dovercourt Village,43.669005,-79.442259,Parallel,43.669543,-79.438701,Middle Eastern Restaurant
2,Dovercourt Village,43.669005,-79.442259,Planet Fitness Toronto Galleria,43.667588,-79.442574,Gym / Fitness Center
3,Dovercourt Village,43.669005,-79.442259,FreshCo,43.667918,-79.440754,Supermarket
4,Dovercourt Village,43.669005,-79.442259,Happy Bakery & Pastries,43.667050,-79.441791,Bakery


In [183]:
HP_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Brockton,21,21,21,21,21,21
Dovercourt Village,17,17,17,17,17,17
Dufferin,17,17,17,17,17,17
Exhibition Place,21,21,21,21,21,21
High Park,23,23,23,23,23,23
Little Portugal,64,64,64,64,64,64
Parkdale,13,13,13,13,13,13
Parkdale Village,21,21,21,21,21,21
Roncesvalles,13,13,13,13,13,13


In [184]:
print('There are {} uniques categories.'.format(len(HP_venues['Venue Category'].unique())))

There are 85 uniques categories.


In [186]:
# one hot encoding
HP_onehot = pd.get_dummies(HP_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
HP_onehot['Neighborhood'] = HP_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [HP_onehot.columns[-1]] + list(HP_onehot.columns[:-1])
HP_onehot = HP_onehot[fixed_columns]

HP_onehot.head()

,Neighborhood,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bank,Bar,Bookstore,Boutique,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Climbing Gym,Cocktail Bar,Coffee Shop,Convenience Store,Cuban Restaurant,Cupcake Shop,Dessert Shop,Diner,Discount Store,Dog Run,Eastern European Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Ice Cream Shop,Indie Movie Theater,Italian Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Liquor Store,Mac & Cheese Joint,Malay Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Music Venue,New American Restaurant,Park,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pizza Place,Pub,Record Shop,Restaurant,Salon / Barbershop,Sandwich Place,Smoothie Shop,Southern / Soul Food Restaurant,Sports Bar,Stadium,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Dovercourt Village,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Dovercourt Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Dovercourt Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Dovercourt Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,Dovercourt Village,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [188]:
HP_onehot.shape

(373, 86)

In [189]:
HP_grouped = HP_onehot.groupby('Neighborhood').mean().reset_index()
HP_grouped

,Neighborhood,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bank,Bar,Bookstore,Boutique,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Climbing Gym,Cocktail Bar,Coffee Shop,Convenience Store,Cuban Restaurant,Cupcake Shop,Dessert Shop,Diner,Discount Store,Dog Run,Eastern European Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Ice Cream Shop,Indie Movie Theater,Italian Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Liquor Store,Mac & Cheese Joint,Malay Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Music Venue,New American Restaurant,Park,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pizza Place,Pub,Record Shop,Restaurant,Salon / Barbershop,Sandwich Place,Smoothie Shop,Southern / Soul Food Restaurant,Sports Bar,Stadium,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Brockton,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.095238,0.000000,0.000000,0.047619,0.095238,0.000000,0.047619,0.047619,0.00000,0.142857,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.047619,0.047619,0.047619,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
1,Dovercourt Village,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.117647,0.058824,0.058824,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.00000,0.000000,0.058824,0.000000,0.000000,0.058824,0.000000,0.058824,0.000000,0.058824,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.117647,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
2,Dufferin,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.117647,0.058824,0.058824,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.00000,0.000000,0.058824,0.000000,0.000000,0.058824,0.000000,0.058824,0.000000,0.058824,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.117647,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
3,Exhibition Place,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.095238,0.000000,0.000000,0.047619,0.095238,0.000000,0.047619,0.047619,0.00000,0.142857,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.047619,0.047619,0.047619,0.000000

In [190]:
HP_grouped.shape

(13, 86)

In [191]:
num_top_venues = 5

for hood in HP_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = HP_grouped[HP_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Brockton----
            venue  freq
0     Coffee Shop  0.14
1            Café  0.10
2  Breakfast Spot  0.10
3             Gym  0.05
4         Stadium  0.05


----Dovercourt Village----
                       venue  freq
0                Supermarket  0.12
1                     Bakery  0.12
2  Middle Eastern Restaurant  0.06
3       Fast Food Restaurant  0.06
4                       Café  0.06


----Dufferin----
                       venue  freq
0                Supermarket  0.12
1                     Bakery  0.12
2  Middle Eastern Restaurant  0.06
3       Fast Food Restaurant  0.06
4                       Café  0.06


----Exhibition Place----
            venue  freq
0     Coffee Shop  0.14
1            Café  0.10
2  Breakfast Spot  0.10
3             Gym  0.05
4         Stadium  0.05


----High Park----
                venue  freq
0  Mexican Restaurant  0.09
1                 Bar  0.09
2                Café  0.09
3               Diner  0.04
4        Antique Shop  0.04


----Little

In [192]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [193]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = HP_grouped['Neighborhood']

for ind in np.arange(HP_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(HP_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brockton,Coffee Shop,Café,Breakfast Spot,Gym,Burrito Place,Grocery Store,Furniture / Home Store,Gym / Fitness Center,Italian Restaurant,Falafel Restaurant
1,Dovercourt Village,Bakery,Supermarket,Brewery,Fast Food Restaurant,Music Venue,Park,Pet Store,Pharmacy,Discount Store,Liquor Store
2,Dufferin,Bakery,Supermarket,Brewery,Fast Food Restaurant,Music Venue,Park,Pet Store,Pharmacy,Discount Store,Liquor Store
3,Exhibition Place,Coffee Shop,Café,Breakfast Spot,Gym,Burrito Place,Grocery Store,Furniture / Home Store,Gym / Fitness Center,Italian Restaurant,Falafel Restaurant
4,High Park,Mexican Restaurant,Café,Bar,Bookstore,Music Venue,Diner,Park,Cajun / Creole Restaurant,Fast Food Restaurant,Italian Restaurant
5,Little Portugal,Bar,Café,Restaurant,Coffee Shop,French Restaurant,Men's Store,Cocktail Bar,Pizza Place,Bakery,Asian Restaurant
6,Parkdale,Gift Shop,Coffee Shop,Breakfast Spot,Burger Joint,Dessert Shop,Piano Bar,Movie Theater,Bar,Dog Run,Eastern European Restaurant
7,Parkdale Village,Coffee Shop,Café,Breakfast Spot,Gym,Burrito Place,Grocery Store,Furniture / Home Store,Gym / Fitness Center,Italian Restaurant,Falafel Restaurant
8,Roncesvalles,Gift Shop,Coffee Shop,Breakfast Spot,Burger Joint,Dessert Shop,Piano Bar,Movie Theater,Bar,Dog Run,Eastern European Restaurant
9,Runnymede,Coffee Shop,Café,Diner,Italian Restaurant,Restaurant,Sushi Restaurant,Pizza Place,Indie Movie Theater,Falafel Restaurant,Pub


In [194]:
# set number of clusters
kclusters = 5

HP_grouped_clustering = HP_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(HP_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 3, 3, 1, 4, 0, 2, 1, 2, 0], dtype=int32)

In [197]:
HP_merged = HighPark

# add clustering labels
HP_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
HP_merged = HP_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

HP_merged.head() # check the last columns!

/opt/ibm/conda/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
52,M6H,West Toronto,Dovercourt Village,43.669005,-79.442259,1,Bakery,Supermarket,Brewery,Fast Food Restaurant,Music Venue,Park,Pet Store,Pharmacy,Discount Store,Liquor Store
53,M6H,West Toronto,Dufferin,43.669005,-79.442259,3,Bakery,Supermarket,Brewery,Fast Food Restaurant,Music Venue,Park,Pet Store,Pharmacy,Discount Store,Liquor Store
64,M6J,West Toronto,Little Portugal,43.647927,-79.419750,3,Bar,Café,Restaurant,Coffee Shop,French Restaurant,Men's Store,Cocktail Bar,Pizza Place,Bakery,Asian Restaurant
65,M6J,West Toronto,Trinity,43.647927,-79.419750,1,Bar,Café,Restaurant,Coffee Shop,French Restaurant,Men's Store,Cocktail Bar,Pizza Place,Bakery,Asian Restaurant
76,M6K,West Toronto,Brockton,43.636847,-79.428191,4,Coffee Shop,Café,Breakfast Spot,Gym,Burrito Place,Grocery Store,Furniture / Home Store,Gym / Fitness Center,Italian Restaurant,Falafel Restaurant


In [199]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(HP_merged['Latitude'], HP_merged['Longitude'], HP_merged['Neighbourhood'], HP_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [200]:
HP_merged.loc[HP_merged['Cluster Labels'] == 0, HP_merged.columns[[1] + list(range(5, HP_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
77,West Toronto,0,Coffee Shop,Café,Breakfast Spot,Gym,Burrito Place,Grocery Store,Furniture / Home Store,Gym / Fitness Center,Italian Restaurant,Falafel Restaurant
134,West Toronto,0,Gift Shop,Coffee Shop,Breakfast Spot,Burger Joint,Dessert Shop,Piano Bar,Movie Theater,Bar,Dog Run,Eastern European Restaurant
135,West Toronto,0,Gift Shop,Coffee Shop,Breakfast Spot,Burger Joint,Dessert Shop,Piano Bar,Movie Theater,Bar,Dog Run,Eastern European Restaurant
146,West Toronto,0,Coffee Shop,Café,Diner,Italian Restaurant,Restaurant,Sushi Restaurant,Pizza Place,Indie Movie Theater,Falafel Restaurant,Pub


In [201]:
HP_merged.loc[HP_merged['Cluster Labels'] == 1, HP_merged.columns[[1] + list(range(5, HP_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
52,West Toronto,1,Bakery,Supermarket,Brewery,Fast Food Restaurant,Music Venue,Park,Pet Store,Pharmacy,Discount Store,Liquor Store
65,West Toronto,1,Bar,Café,Restaurant,Coffee Shop,French Restaurant,Men's Store,Cocktail Bar,Pizza Place,Bakery,Asian Restaurant
124,West Toronto,1,Mexican Restaurant,Café,Bar,Bookstore,Music Venue,Diner,Park,Cajun / Creole Restaurant,Fast Food Restaurant,Italian Restaurant


In [202]:
HP_merged.loc[HP_merged['Cluster Labels'] == 3, HP_merged.columns[[1] + list(range(5, HP_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,West Toronto,3,Bakery,Supermarket,Brewery,Fast Food Restaurant,Music Venue,Park,Pet Store,Pharmacy,Discount Store,Liquor Store
64,West Toronto,3,Bar,Café,Restaurant,Coffee Shop,French Restaurant,Men's Store,Cocktail Bar,Pizza Place,Bakery,Asian Restaurant


In [203]:
HP_merged.loc[HP_merged['Cluster Labels'] == 4, HP_merged.columns[[1] + list(range(5, HP_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
76,West Toronto,4,Coffee Shop,Café,Breakfast Spot,Gym,Burrito Place,Grocery Store,Furniture / Home Store,Gym / Fitness Center,Italian Restaurant,Falafel Restaurant
145,West Toronto,4,Coffee Shop,Café,Diner,Italian Restaurant,Restaurant,Sushi Restaurant,Pizza Place,Indie Movie Theater,Falafel Restaurant,Pub


# Thank you for reviwing my work!